# Matrice de Rotatie Optima Folosind SVD

In cele ce urmeaza, vom vedea cum putem calcula o matrice de rotatie folosind algoritmul Kabsch. Acest algoritm ne da cea mai buna astfel de matrice, pe scurt matricea de rotatie optima.

### De unde necesitatea unei astfel de matrice?

O astfel de matrice este des intalnita in problema ortogonalitatii Procust. De unde aceasta denumire, si ce reprezinta de fapt aceasta noua problema?

De multe ori intalnita si sub denumirea de superimpositie Procust (Procustes superimposition), acest nume pompos nu trebuie sa ne sperie deloc. Procust este un bandit din Grecia Antica, bine cunoscut pentru faptul ca isi tortura victimele prin alungirea membrelor sau trunchiului de asa maniera incat sa ocupe intreaga lungime a patului sau.

Acest mit al lui Procust este regasit in multe zone, sub diferite aspecte. Mai mult ca sigur ati auzit de Patul lui Procust de Camil Petrescu, pe care o recomand cu drag ca si lectura suplimentara, insa nu acesta este scopul cursului. O alta carte, mult mai interesanta decat cea amintita anterior, se poate gasi aici: https://en.wikipedia.org/wiki/The_Bed_of_Procrustes.

### Deci, care este problema lui Procust?

Problema lui Procust este o problema de aproximare a matricilor, de a gasi ceva "asemanator" intre 2 matrici. Scopul acestei probleme este de a gasi o matrice ortogonala, $\Omega$, care sa "mapeze" o matrice data A, pe o alta matrice data B.

Algoritmul Kabsch nu este altceva decat un caz mai specific al problemei lui Procust, reprezentand anume cazul in care apar fenomenele de translatie si rotatie.

Haideti sa consideram urmatoarea problema: la un studiu participa 7 barbati si 7 femei; pentru fiecare dintre acestia, avem 3 detalii importante: varsta, inaltime si greutate. Datele sunt reprezentate pe 3 coloane, in doua matrici diferite: una pentru barbati, si cealalta pentru femei.


In [3]:
Men = [25 174 79; 33 181 76; 27 178 87; 57 178 91; 68 167 74; 41 186 86; 38 184 90]

Men =

    25   174    79
    33   181    76
    27   178    87
    57   178    91
    68   167    74
    41   186    86
    38   184    90



In [4]:
Women = [67 165 68; 35 171 65; 44 156 49; 53 169 55; 81 164 91; 22 178 63; 33 173 82]

Women =

    67   165    68
    35   171    65
    44   156    49
    53   169    55
    81   164    91
    22   178    63
    33   173    82



Aplicarea algoritmului Kabsch are 3 pasi, pentru aflarea matricei de rotatie:

### 1) Translatia matricelor

Primul pas consta in a realiza o translatie pentru fiecare matrice, in functie de centroidul corespunzator.
Ce este acela un centroid? In linii mari, centroidul este un punct in spatiu (atentie, aici depinde de dimensiunea spatiului, care poate fi bi sau tridimensional, sau chiar n-dimensional), care ar reprezenta centrul de greutate al unui poligon (sau corp geometric) format prin unirea tutror punctelor, astfel incat sa obtinem o figura/ un corp convex. Cu alte cuvinte, se face referire la suma minima a distantelor euclidiene intre fiecare punct si centroid, la patrat (aka metoda celor mai mici patrate).

Vom nota cei doi centroizi cu Cm, pentru barbati, si Cw pentru femei. Obtinem astfel:

In [5]:
Cm = [sum(Men(:, 1)) sum(Men(:, 2)) sum(Men(:, 3))] / size(Men, 1)

Cm =

    41.286   178.286    83.286



In [6]:
Cw = [sum(Women(:, 1)) sum(Women(:, 2)) sum(Women(:, 3))] / size(Women, 1)

Cw =

    47.857   168.000    67.571



Acum vom face translatia in functie de centroizii obtinuti:

In [11]:
TMen = Men - ones(size(Men),1) * Cm

TMen =

  -16.28571   -4.28571   -4.28571
   -8.28571    2.71429   -7.28571
  -14.28571   -0.28571    3.71429
   15.71429   -0.28571    7.71429
   26.71429  -11.28571   -9.28571
   -0.28571    7.71429    2.71429
   -3.28571    5.71429    6.71429



In [12]:
TWomen = Women - ones(size(Women),1) * Cw

TWomen =

   19.14286   -3.00000    0.42857
  -12.85714    3.00000   -2.57143
   -3.85714  -12.00000  -18.57143
    5.14286    1.00000  -12.57143
   33.14286   -4.00000   23.42857
  -25.85714   10.00000   -4.57143
  -14.85714    5.00000   14.42857



Dupa ce am reusit sa translatam cele doua matrici, trecem la pasul 2, anume gasirea matricei de covarianta.

### 2) Constructia matricei de covarianta

Matricea de covarianta este, pe scurt, o matrice care prezinta "volatilitatea" dintre doua seturi de date. Ea poate fi calculata cu diferite formule, dar noi avem nevoie doar de una scurta si simpla, care exceleaza in problema noastra:

$H = P^T * Q$

In [16]:
H = TMen' * TWomen

H =

   872.286    85.000   661.857
  -775.714   175.000  -217.143
  -440.714    52.000  -282.143



De ce alegem o formula de genul $P^T * Q$ si nu una de genul $Q^T * P$? Raspunsul este destul de intuitiv: cea de-a doua formula nu reprezinta altceva decat $H^T$

$(P^T * Q)^T = Q^T * (P^T)^T = Q^T * P$ (reguli esentiale din liceu :) )

Prin urmare, nu conteaza extrem de mult felul in care dorim sa aflam matricea H, aceasta fiind aceeasi indiferent de alegere, doar ca una este transpusa in comparatie cu cealalta.

O mica remarca in legatura cu H este faptul ca ea trebuie sa fie in general o matrice patratica. Ultimul pas consta in calcularea matricei de rotatie efectiva, folosindu-ne strict de matricea de corelatie gasita la pasul anterior.

### 3)Calculul matricei de rotatie optima 

Pentru calculul acesteia, putem folosi o formula "clasica", gasita de asemenea pe internet:

$R = (H^T * H)^{1/2} * H^{-1}$

Desigur, formula, la prima vedere, nu pare super intuitiva. Pe langa faptul ca prezinta si anumite probleme, cum ar fi faptul ca det(H) poate fi egal cu 0. Adica matricea H este neinversabila, deci nu putem calcula R cu aceasta metoda intotdeauna.

Care ramane solutia acestei probleme? Descompunerea matricei H in valori singulare :(

Vom avea niste linii suplimentare de scris, insa orice lucru in plus are un cost suplimentar in viata. Atat timp cat putem rezolva o problema destul mare, nu ar trebui sa ne para rau.

Prin urmare, facem descompunerea lui H:

In [17]:
[U S V] = svd(H)

U =

  -0.7536293  -0.5901396   0.2894445
   0.5467367  -0.8072476  -0.2223292
   0.3648587  -0.0093039   0.9310164

S =

Diagonal Matrix

   1436.977          0          0
          0    308.881          0
          0          0     43.574

V =

  -0.864516   0.374007   0.335753
   0.035208  -0.621321   0.782765
  -0.501370  -0.688534  -0.523974



Apoi, trebuie sa ne asiguram ca rotatia pe care vrem sa o facem este intr-un sens bine cunoscut. Conventional, vom alege acest sens spre dreapta. Desi pare putin ambiguu ceea ce facem aici, o sa explicam imediat ce si cum (in linii mari, fara concepte prea hardcore)

In [18]:
d = sign(det(V*U'))

d = -1


Acum vin intrebarile: ce s-a intamplat aici? Din moment ce ne intereseaza in ce directie vrem sa facem rotatia, lucrul care ne intereseaza, de fapt, este SENSUL. Cum putem exprima sensul? Fie pozitiv semnul "+", fie negativ, semnul "-". Functia sign ne da semnul determinantului de mai sus. Tot din liceu, avem niste relatii si pentru determinanti, care ne fac sa inteleg niste chestii mult mai usor:

$det(A * B) = det(A) * det(B)$

Acestea fiind spuse, putem simplu sa facem legatura: pe noi ne intereseaza, de fapt, semnul fiecarui determinant in parte: det(V) si det(U<sup>T</sup>). Totul s-ar reduce dupa la produsul semnelor, studiat cel mai probabil in gimnaziu.

In final, matricea de rotatie va avea urmatoarea forma:

In [21]:
D = eye(size(H))
D(end, end) = d
R = V * D * U'

D =

Diagonal Matrix

   1   0   0
   0   1   0
   0   0   1

D =

Diagonal Matrix

   1   0   0
   0   1   0
   0   0  -1

R =

   0.33363  -0.69993  -0.63150
   0.11357   0.69484  -0.71014
   0.93584   0.16521   0.31131



Fun fact: determinantul matricei de rotatie este mereu egal cu 1.

In [22]:
det(R)

ans =  1.00000


Pentru olimpicii care doresc sa aprofundeze si care nu se multumesc cu putin, avem mai multe detalii la acest link: https://math.nist.gov/~JBernal/kujustf.pdf